In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
import tensorflow as tf
import tensornetwork
from tensornetwork import ncon
import seaborn as sns
net = tensornetwork.TensorNetwork()
sess = tf.InteractiveSession()

# Matrix Product State (MPS) o tensor train (TT)
​
<img src="http://tensornetwork.org/mps/mpstt_diagram.png" width="400">
​
*    Es la factorizacion de un tensor con N indices en un producto tipo cadena lineal de tensores de tres índices. 
​
*    Es un caso especial de una red tensorial de árbol.
​
*    <br>
<img src="https://tensornetwork.org/mps/bond_external_dim.png" width="300">
<br>
$d^N$ = Numero de parametros de tensor $T$ de orden $N$ y dimension local $d$ 
<br>
$Ndm^2$ $\ge$ Numero de parametros de representacion MPS de $T$ (seria igual si el rango interno fuera el mismo en todo el MPS)

# Descomposicion MPS/TT-SVD

<img src="https://drive.google.com/uc?export=view&id=1TOe1YsvrTCLGHX3M5k7xPPSwmi16bbTi" width="500">
<br>
<img src="https://drive.google.com/uc?export=view&id=1DSBfh1BoXruSJlJbdSMQ2DQKKw5zPBih" width="400">


In [2]:
##### Descomposicion de tensor de 4 indices a MPS usando SVD truncado con ortogonalidad izquierda
d = 30 # dimension externa
H0 = net.add_node(np.random.rand(d,d,d,d)) 
H0 = net.add_node(H0.tensor/tf.norm(H0.tensor))
eps = 0.1
deltaval=eps/np.sqrt(4-1)

# primera descomposicion
u0,s,vh,trun_err = net.split_node_full_svd(H0,[H0[0]],[H0[1],H0[2],H0[3]],None,deltaval)
H1 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2,-3,-4]]))
print("u0: ",u0.tensor.shape,"s0: ",s.tensor.shape,"vh0: ",vh.tensor.shape)
print("H1: ",H1.tensor.shape)
# segunda descomposicion
u1,s,vh,trun_err = net.split_node_full_svd(H1,[H1[0],H1[1]],[H1[2],H1[3]],None,deltaval)
H2 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2,-3]]))
print("u1: ",u1.tensor.shape,"s1: ",s.tensor.shape,"vh1: ",vh.tensor.shape)
print("H2: ",H2.tensor.shape)
# tercera descomposicion
u2,s,vh,trun_err = net.split_node_full_svd(H2,[H2[0],H2[1]],[H2[2]],None,deltaval)
H3 = net.add_node(ncon([s.tensor,vh.tensor],[[-1,1],[1,-2]])) 
print("u2: ",u2.tensor.shape,"s2: ",s.tensor.shape,"vh2: ",vh.tensor.shape)
print("H3: ",H3.tensor.shape)

#verificar
H0recovered = net.add_node(ncon([u0.tensor,u1.tensor,u2.tensor,H3.tensor],[[-1,1],[1,-2,2],[2,-3,3],[3,-4]]))
print("totErr = ",(tf.norm(H0recovered.tensor-H0.tensor)/tf.norm(H0.tensor)).eval())

u0:  (30, 30) s0:  (30, 30) vh0:  (30, 30, 30, 30)
H1:  (30, 30, 30, 30)
u1:  (30, 30, 673) s1:  (673, 673) vh1:  (673, 30, 30)
H2:  (673, 30, 30)
u2:  (673, 30, 30) s2:  (30, 30) vh2:  (30, 30)
H3:  (30, 30)
totErr =  0.05764783525841739


# Operaciones elementales con MPS/TT

  
###    Producto interno o superposicion de dos MPS/TT
<img src="http://tensornetwork.org/mps/TdotW.png" width="200">
<br>
<img src="http://tensornetwork.org/mps/TW_MPSTT.png" width="250">
<br>
<img src="http://tensornetwork.org/mps/InnerMPSTT.png" width="250">
  
###    Compresion o redondeo
  
<img src="http://tensornetwork.org/mps/mpstt_compress.png" width="400">

# Matrix Product Operator (MPO)

Dado un conjunto de $L$ espacios de Hilbert $\mathcal{H}_i$, $i \in [1,L]$, de dimension $d_i$ cada uno y un operador $\hat{H}$ que actua sobre el espacio del producto tensorial $\mathcal{H}$ = $\otimes \mathcal{H}_i$, podemos escribir el operador como:

<font size="5"><center>$\hat{H} = \sum_{\vec{\sigma} \vec{\sigma}'}c^{\vec{\sigma}'}_{\vec{\sigma}}|\vec{\sigma}'\rangle \langle \vec{\sigma}|$</center></font>

Donde $|\vec{\sigma}'\rangle$ enumera las bases estados de $\mathcal{H}$ y $\langle \vec{\sigma}|$ las bases estado del espacio dual de $\mathcal{H}$. Podemos descomponer cada vector base $|\vec{\sigma}'\rangle$ como el producto tensorial de los vectores base en los espacios locales individuales como 

<font size="5"><center>$|\vec{\sigma}'\rangle = \otimes_{i=1}^L |\sigma'_i\rangle = |\sigma'\rangle = |\sigma'_1,...,\sigma'_L \rangle$</center></font>

lo que lleva a 

<font size="5"><center>$\hat{H} = \sum_{\sigma'_1 \sigma_1}...\sum_{\sigma'_L \sigma_L}c^{\sigma'_1...\sigma'_L}_{\sigma_1...\sigma_L} |\sigma'_1,...,\sigma'_L \rangle \langle\sigma_1,...,\sigma_L|$</center></font>


![](https://drive.google.com/uc?export=view&id=16qCO97OliTfbz8e-W2ZdZlmkSd3FHm32)


El coeficiente $c$ ahora puede descomponerse como un conjunto de productos matriciales. Es decir, en cada sitio $i$ y para cada combinación de estados locales $\{|\sigma'_i,\sigma_i\}$, introducimos un conjunto de matrices $(W^{w_{i-1}\sigma'_i}_{[i]w_i\sigma_i})$ con la propiedad de que su producto matriz-matriz es igual a un elemento específico del tensor $c$:

<font size="5"><center>$\sum_{\mu,\nu,...,\lambda} W^{\sigma'_1}_{\mu \sigma_1}W^{\mu \sigma'_2}_{\nu \sigma_2}...W^{\lambda \sigma'_L}_{ \sigma_L} = c^{\sigma'_1...\sigma'_L}_{\sigma_1...\sigma_L}$</center></font>

![](https://drive.google.com/uc?export=view&id=1bgbFqgrq7RdUYFwCn6LAHhaMEj77nNwG)



## Aplicando un MPO a un estado canonico mixto

¿Como actua un MPO sobre un MPS en representacion canonica mixta con respecto a un sitio $l$?
<br>
![](https://drive.google.com/uc?export=view&id=1JGF2t0P9GuqXW6fOnjlKRXi56jDzWeoi)
<br>
Acá $\{|a\rangle\}$ forma una base para la representacion canonica mixta. Exprese el operador en esta base:
<br>

<font size="5"><center>$\hat{O} = |a'>O^{a'}_{a}<a|$</center></font> 

con elementos matriciales

<font size="5"><center>$O^{a'}_{a} = <a'|\hat{O}|a>$ = $L^{\alpha'_l}_{[l-1]\mu_l \alpha_l} W^{\mu_l \sigma'_l \nu_l}_{[l]\sigma_l} R^{\beta'_l}_{[l+1]\nu_l \beta_l}$</center></font>
<br>
![](https://drive.google.com/uc?export=view&id=1YWOacMc3iokE33IpvHHe37NlYu8AlEDT)
<br>

Finalmente,
<br>

<font size="5"><center>$|\psi'> = \hat{O}|\psi> = |a'>A^{'a'}$ $\hspace{2mm}$ donde  $\hspace{2mm}$ $A^{'a'} = O^{a'}_{a}A^a$</center></font>
<br>
![](https://drive.google.com/uc?export=view&id=10P3mFfgmQX2psG-6dMqTnv2fcoViyY9i)
<br>